In [178]:
from simtk import unit
from openmmtools import testsystems, cache
from openmmtools.mcmc import GHMCMove, MCMCSampler, MCRotationMove, BaseIntegratorMove, IntegratorMoveError
from openmmtools.states import ThermodynamicState, SamplerState, CompoundThermodynamicState
from simtk.openmm import CompoundIntegrator
from openmmtools import alchemy
from simtk import unit
from openmmtools.utils import RestorableOpenMMObject
from openmmtools.integrators import *
import numpy as np
import copy, sys, time, os
import nglview
import logging
from simtk import openmm
import parmed
from openmmtools.utils import SubhookedABCMeta, Timer, RestorableOpenMMObject
from openmmtools import testsystems, alchemy
import netCDF4 as nc
from blues.integrators import AlchemicalExternalLangevinIntegrator
import mdtraj
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
np.random.RandomState(seed=3134)
logging.basicConfig(format='%(asctime)s | %(levelname)s : %(message)s', level=logging.INFO, stream=sys.stdout)

# Below try it inside classes

In [117]:
class DummySimulation(object):
    def __init__(self, integrator, system, topology):
        self.integrator = integrator
        self.topology = topology
        self.system = system

In [237]:
class NCMCMove(BaseIntegratorMove):

    def __init__(self,
                 timestep=1.0 * unit.femtosecond,
                 collision_rate=10.0 / unit.picoseconds,
                 n_steps=1000,
                 temperature=300 * unit.kelvin,
                 reassign_velocities=True,
                 **kwargs):
        super(NCMCMove, self).__init__(n_steps=n_steps, **kwargs)
        self.timestep = timestep
        self.temperature = temperature
        self.collision_rate = collision_rate
        self.n_accepted = 0  # Number of accepted steps.
        self.n_proposed = 0  # Number of attempted steps.
        self.random_state = np.random.RandomState(1)

    @property
    def fraction_accepted(self):
        """Ratio between accepted over attempted moves (read-only).

        If the number of attempted steps is 0, this is numpy.NaN.

        """
        if self.n_proposed == 0:
            return np.NaN
        # TODO drop the casting when stop Python2 support
        return float(self.n_accepted) / self.n_proposed

    @property
    def statistics(self):
        """The acceptance statistics as a dictionary."""
        return dict(n_accepted=self.n_accepted, n_proposed=self.n_proposed)

    @statistics.setter
    def statistics(self, value):
        self.n_accepted = value['n_accepted']
        self.n_proposed = value['n_proposed']

    def reset_statistics(self):
        """Reset the internal statistics of number of accepted and attempted moves."""
        self.n_accepted = 0
        self.n_proposed = 0
        
    def _create_dummy_simulation(self, integrator, system, topology):
        """
        Generate a dummy Simulation object because the Reporter
        expects an `openmm.Simulation` object in order to report information
        from it's respective attached integrator/
        """
        return DummySimulation(integrator, system, topology) 
    
    def _before_integration(self, context, integrator, thermodynamic_state):
        """Execute code after Context creation and before integration."""
        
        return self._create_dummy_simulation(integrator,
                                            thermodynamic_state.get_system(), 
                                            thermodynamic_state.topology)
    
    def getMasses(self, atom_subset, topology):
        """
        Returns a list of masses of the specified ligand atoms.
        Parameters
        ----------
        topology: parmed.Topology
            ParmEd topology object containing atoms of the system.
        Returns
        -------
        masses: 1xn numpy.array * simtk.unit.dalton
            array of masses of len(self.atom_indices), denoting
            the masses of the atoms in self.atom_indices
        totalmass: float * simtk.unit.dalton
            The sum of the mass found in masses
        """
        atoms = [ list(topology.atoms())[i] for i in atom_subset]
        masses = unit.Quantity(np.zeros([int(len(atoms)), 1], np.float32), unit.dalton)
        for idx, atom in enumerate(atoms):
            masses[idx] = atom.element._mass
        totalmass = masses.sum()
        return masses, totalmass

    def getCenterOfMass(self, positions, masses):
        """Returns the calculated center of mass of the ligand as a numpy.array
        Parameters
        ----------
        positions: nx3 numpy array * simtk.unit compatible with simtk.unit.nanometers
            ParmEd positions of the atoms to be moved.
        masses : numpy.array
            numpy.array of particle masses
        Returns
        -------
        center_of_mass: numpy array * simtk.unit compatible with simtk.unit.nanometers
            1x3 numpy.array of the center of mass of the given positions
        """
        coordinates = np.asarray(positions._value, np.float32)
        center_of_mass = parmed.geometry.center_of_mass(coordinates, masses) * positions.unit
        return center_of_mass
    
    def rotate(self, context, thermodynamic_state, sampler_state):
        print('Performing rotation...')
        atom_subset = thermodynamic_state.alchemical_atoms
        context_state = context.getState(getPositions=True, getVelocities=True, getEnergy=True,
                                 enforcePeriodicBox=thermodynamic_state.is_periodic)
        all_positions = context_state.getPositions(asNumpy=True)
        sub_positions = all_positions[atom_subset]
        
        masses, totalmass = self.getMasses(atom_subset, thermodynamic_state.topology)
        center_of_mass = self.getCenterOfMass(sub_positions, masses)
        reduced_pos = sub_positions - center_of_mass

        # Define random rotational move on the ligand
        rand_quat = mdtraj.utils.uniform_quaternion(size=None, random_state=self.random_state)
        rand_rotation_matrix = mdtraj.utils.rotation_matrix_from_quaternion(rand_quat)
        #multiply lig coordinates by rot matrix and add back COM translation from origin
        rot_move = np.dot(reduced_pos, rand_rotation_matrix) * sub_positions.unit + center_of_mass

        # Update ligand positions in sampler_state and context
        for index, atomidx in enumerate(atom_subset):
            all_positions[atomidx] = rot_move[index]
        sampler_state.positions = all_positions
        sampler_state.apply_to_context(context, ignore_velocities=True)
      
        return context
    
    def apply(self, thermodynamic_state, sampler_state, integrator, reporter, alch=True):
        """Propagate the state through the integrator.
        This updates the SamplerState after the integration. It also logs
        benchmarking information through the utils.Timer class.
        Parameters
        ----------
        thermodynamic_state : openmmtools.states.ThermodynamicState
           The thermodynamic state to use to propagate dynamics.
        sampler_state : openmmtools.states.SamplerState
           The sampler state to apply the move to. This is modified.
        See Also
        --------
        openmmtools.utils.Timer
        """
        move_name = self.__class__.__name__  # shortcut
        timer = Timer()

        # Check if we have to use the global cache.
        if self.context_cache is None:
            context_cache = cache.global_context_cache
        else:
            context_cache = self.context_cache

        # Create context.
        timer.start("{}: Context request".format(move_name))
        context, integrator = context_cache.get_context(thermodynamic_state, integrator)
        print(integrator)
        timer.stop("{}: Context request".format(move_name))
        logger.debug("{}: {} \t Context obtained, platform is {}".format(
            move_name, integrator, context.getPlatform().getName()))
        
        
        if alch:
            rotation_step = int(n_steps/2)
        
        
        # Perform the integration.
        for attempt_counter in range(self.n_restart_attempts + 1):

            # If we reassign velocities, we can ignore the ones in sampler_state.
            sampler_state.apply_to_context(context, ignore_velocities=self.reassign_velocities)
            if self.reassign_velocities:
                context.setVelocitiesToTemperature(thermodynamic_state.temperature)

            # Subclasses may implement _before_integration().
            simulation = self._before_integration(context, integrator, thermodynamic_state)

            try:
                # Run dynamics.
                timer.start("{}: step({})".format(move_name, self.n_steps))
                
                #NML: Do in 1 steps for debugging
                for n in range(self.n_steps+1):
                    integrator.step(1)
                 
                    if alch:
                        step = integrator.getGlobalVariableByName('step')
                        alch_lambda = integrator.getGlobalVariableByName('lambda')
                        lambda_step = integrator.getGlobalVariableByName('lambda_step')
                        protocol_work = integrator.getGlobalVariableByName('protocol_work')

                        thermodynamic_state.set_alchemical_variable('lambda', alch_lambda)
                        thermodynamic_state.apply_to_context(context)

                        #lambda_sterics = context.getParameter('lambda_sterics')
                        #lambda_electrostatics = context.getParameter('lambda_electrostatics')
                        
                        if n == rotation_step:
                            context = self.rotate(context, thermodynamic_state, sampler_state)
                            
                        # Get the context state for reporting
                        context_state = context.getState(getPositions=True, getVelocities=True, getEnergy=True,
                                                 enforcePeriodicBox=thermodynamic_state.is_periodic)
                        print('IntgrParameters:', 'Step:', step, 'PE:', context_state.getPotentialEnergy(), 'Work:', protocol_work, 'Lambda: ', alch_lambda, 'Lambda Step: ', lambda_step) 
                        #print('\t'+'ContextParameters:', lambda_step, 'Sterics:', lambda_sterics, 'Elec:', lambda_electrostatics)     
                        
                        if step == n_steps:
                            break
                
                    else:
                        # Get the context state for reporting
                        context_state = context.getState(getPositions=True, getVelocities=True, getEnergy=True,
                                                 enforcePeriodicBox=thermodynamic_state.is_periodic)
                        #print('Pos:', context_state.getPositions()[0])
                        print('Step:', n, 'PE:', context_state.getPotentialEnergy())
                        
  
                    reporter.report(simulation, context_state)
                
            except Exception as e:
                print(e)
                
                # Catches particle positions becoming nan during integration.
                restart = True
            else:
                timer.stop("{}: step({})".format(move_name, self.n_steps))

                # We get also velocities here even if we don't need them because we
                # will recycle this State to update the sampler state object. This
                # way we won't need a second call to Context.getState().
                context_state = context.getState(getPositions=True, getVelocities=True, getEnergy=True,
                                                 enforcePeriodicBox=thermodynamic_state.is_periodic)

                # Check for NaNs in energies.
                potential_energy = context_state.getPotentialEnergy()
                restart = np.isnan(potential_energy.value_in_unit(potential_energy.unit))

            # Restart the move if we found NaNs.
            if restart:
                err_msg = ('Potential energy is NaN after {} attempts of integration '
                           'with move {}'.format(attempt_counter, self.__class__.__name__))

                # If we are on our last chance before crash, try to re-initialize context
                if attempt_counter == self.n_restart_attempts - 1:
                    logger.error(err_msg + ' Trying to reinitialize Context as a last-resort restart attempt...')
                    context.reinitialize()
                    sampler_state.apply_to_context(context)
                    thermodynamic_state.apply_to_context(context)
                # If we have hit the number of restart attempts, raise an exception.
                elif attempt_counter == self.n_restart_attempts:
                    # Restore the context to the state right before the integration.
                    sampler_state.apply_to_context(context)
                    logger.error(err_msg)
                    raise IntegratorMoveError(err_msg, self, context)
                else:
                    logger.warning(err_msg + ' Attempting a restart...')
            else:
                break

        # Subclasses can read here info from the context to update internal statistics.
        self._after_integration(context, thermodynamic_state)

        # Updated sampler state.
        timer.start("{}: update sampler state".format(move_name))
        # This is an optimization around the fact that Collective Variables are not a part of the State,
        # but are a part of the Context. We do this call twice to minimize duplicating information fetched from
        # the State.
        # Update everything but the collective variables from the State object
        sampler_state.update_from_context(context_state, ignore_collective_variables=True)
        # Update only the collective variables from the Context
        sampler_state.update_from_context(context, ignore_positions=True, ignore_velocities=True,
                                          ignore_collective_variables=False)
        timer.stop("{}: update sampler state".format(move_name))

        #timer.report_timing()
        return context
    
    def _get_integrator(self, thermodynamic_state):
        """Implement BaseIntegratorMove._get_integrator()."""
        return LangevinIntegrator(temperature=thermodynamic_state.temperature,
                                          collision_rate=self.collision_rate,
                                          timestep=self.timestep)

    def __getstate__(self):
        if self.context_cache is None:
            context_cache_serialized = None
        else:
            context_cache_serialized = utils.serialize(self.context_cache)
        return dict(n_steps=self.n_steps, context_cache=context_cache_serialized,
                    reassign_velocities=self.reassign_velocities,
                    n_restart_attempts=self.n_restart_attempts)

    def __setstate__(self, serialization):
        self.n_steps = serialization['n_steps']
        self.reassign_velocities = serialization['reassign_velocities']
        self.n_restart_attempts = serialization['n_restart_attempts']
        if serialization['context_cache'] is None:
            self.context_cache = None
        else:
            self.context_cache = utils.deserialize(serialization['context_cache'])


    def _after_integration(self, context, thermodynamic_state):
        """Implement BaseIntegratorMove._after_integration()."""
        integrator = context.getIntegrator()
        try:
            # Accumulate acceptance statistics.
            ghmc_global_variables = {integrator.getGlobalVariableName(index): index
                                 for index in range(integrator.getNumGlobalVariables())}
            n_accepted = integrator.getGlobalVariable(ghmc_global_variables['naccept'])
            n_proposed = integrator.getGlobalVariable(ghmc_global_variables['ntrials'])
            self.n_accepted += n_accepted
            self.n_proposed += n_proposed
        except Exception as e:
            print(e)
            pass


In [238]:
class NCMCSampler(object):


    def __init__(self, thermodynamic_state=None, alch_thermodynamic_state=None, 
                 sampler_state=None, move=None, platform=None, reporter=None, 
                 pdbfile=None, topology=None):
        """
        Create an MCMC sampler.
        Parameters
        ----------
        thermodynamic_state : ThermodynamicState
            The thermodynamic state to simulate
        sampler_state : SamplerState
            The initial sampler state to simulate from.
        platform : simtk.openmm.Platform, optional, default=None
            If specified, this platform will be used
        ncfile : netCDF4.Dataset, optional, default=None
            NetCDF storage file.
        """
        if alch_thermodynamic_state is None:
            raise Exception("'alch_thermodynamic_state' must be specified")
        if thermodynamic_state is None:
            raise Exception("'thermodynamic_state' must be specified")
        if sampler_state is None:
            raise Exception("'sampler_state' must be specified")
            
        self.alch_thermodynamic_state = alch_thermodynamic_state
        self.thermodynamic_state = thermodynamic_state
        
        #NML: Attach topology to thermodynamic_states
        self.alch_thermodynamic_state.topology = topology
        self.thermodynamic_state.topology = topology
        
        self.sampler_state = sampler_state
        self.move = move
        
        # Initialize
        self.iteration = 0
        # For GHMC / Langevin integrator
        self.collision_rate = move.collision_rate
        self.timestep = move.timestep
        self.n_steps = move.n_steps  # number of steps per update
        self.verbose = True
        self.platform = platform

        # For writing trajectory files
        self.reporter = reporter

        

        self._timing = dict()
        
    def _get_alch_integrator(self, alch_thermodynamic_state):
        return AlchemicalExternalLangevinIntegrator(
            alchemical_functions={'lambda_sterics': 'lambda', 
                                  'lambda_electrostatics' : 'lambda'},
            splitting="H V R O R V H",
            temperature=alch_thermodynamic_state.temperature,
            nsteps_neq=self.n_steps,
            timestep=self.timestep,
            nprop=1,
            prop_lambda=0.3)
    
    def _get_integrator(self, thermodynamic_state):
        """Implement BaseIntegratorMove._get_integrator()."""
        return LangevinIntegrator(temperature=thermodynamic_state.temperature,
                                          collision_rate=self.collision_rate,
                                          timestep=self.timestep)
  
    def update(self):
        """
        Update the sampler with one step of sampling.
        """
        if self.verbose:
            print("." * 80)
            print("MCMC sampler iteration %d" % self.iteration)
            
        initial_time = time.time()
        
        # Take some steps with regular LangevinDynamics
        # Create integrator.
        integrator = self._get_integrator(self.thermodynamic_state)
        context = self.move.apply(self.thermodynamic_state,
                                       self.sampler_state, 
                                       integrator,
                                       self.reporter,
                                         alch=False)
        # Update sampler state.
        self.sampler_state.update_from_context(context)
           
        # Take some steps with Alchemical integrator
        alch_integrator = self._get_alch_integrator(self.alch_thermodynamic_state)
        context = self.move.apply(self.alch_thermodynamic_state,
                                       self.sampler_state, 
                                       alch_integrator,
                                       self.reporter,
                                     alch=True)
                


        

        


        final_time = time.time()
        elapsed_time = final_time - initial_time
        self._timing['sample positions'] = elapsed_time 
        
        if self.verbose:
            final_energy = context.getState(getEnergy=True).getPotentialEnergy() * self.thermodynamic_state.beta
            print('Final energy is %12.3f kT' % (final_energy))
            print('elapsed time %8.3f s' % elapsed_time)

    
        # Increment iteration count
        self.iteration += 1

        if self.verbose:
            print("." * 80)

    def run(self, n_iterations=1):
        """
        Run the sampler for the specified number of iterations
        Parameters
        ----------
        niterations : int, optional, default=1
            Number of iterations to run the sampler for.
        """
        for iteration in range(n_iterations):
            self.update()
                        
    def minimize(self,
                 tolerance=1.0 * unit.kilocalories_per_mole / unit.angstroms,
                 max_iterations=100,
                 context_cache=None):
        """Minimize the current configuration.

        Parameters
        ----------
        tolerance : simtk.unit.Quantity, optional
            Tolerance to use for minimization termination criterion (units of
            energy/(mole*distance), default is 1*kilocalories_per_mole/angstroms).
        max_iterations : int, optional
            Maximum number of iterations to use for minimization. If 0, the minimization
            will continue until convergence (default is 100).
        context_cache : openmmtools.cache.ContextCache, optional
            The ContextCache to use for Context creation. If None, the global cache
            openmmtools.cache.global_context_cache is used (default is None).

        """
        if context_cache is None:
            context_cache = cache.global_context_cache

        timer = Timer()

        # Use LocalEnergyMinimizer
        timer.start("Context request")
        integrator = openmm.VerletIntegrator(1.0 * unit.femtosecond)
        context, integrator = context_cache.get_context(self.thermodynamic_state, integrator)
        self.sampler_state.apply_to_context(context)
        logger.debug("LocalEnergyMinimizer: platform is %s" % context.getPlatform().getName())
        logger.debug("Minimizing with tolerance %s and %d max. iterations." % (tolerance, max_iterations))
        timer.stop("Context request")

        timer.start("LocalEnergyMinimizer minimize")
        openmm.LocalEnergyMinimizer.minimize(context, tolerance, max_iterations)
        timer.stop("LocalEnergyMinimizer minimize")
        context_state = context.getState(
            getPositions=True,
            getVelocities=True,
            getEnergy=True,
            enforcePeriodicBox=self.thermodynamic_state.is_periodic)

        potential_energy = context_state.getPotentialEnergy()
        print(potential_energy)
        # Retrieve data.
        self.sampler_state.update_from_context(context)

        #timer.report_timing()

In [282]:
# Define parameters
temperature = 300 * unit.kelvin
collision_rate = 1 / unit.picoseconds
timestep = 4.0 * unit.femtoseconds
n_steps = 4

In [283]:
from blues import utils   
prmtop = utils.get_data_filename('blues', 'tests/data/TOL-parm.prmtop')
inpcrd = utils.get_data_filename('blues', 'tests/data/TOL-parm.inpcrd')
tol = parmed.load_file(prmtop, xyz=inpcrd)
tol.system = tol.createSystem(nonbondedMethod=openmm.app.PME,
                             nonbondedCutoff=8*unit.angstrom,
                             constraints=openmm.app.HBonds,
                             hydrogenMass=3.024*unit.dalton,
                             )

2018-11-16 16:10:55,850 | INFO : Adding bonds...
2018-11-16 16:10:55,854 | INFO : Adding angles...
2018-11-16 16:10:55,857 | INFO : Adding dihedrals...
2018-11-16 16:10:55,857 | INFO : Adding Ryckaert-Bellemans torsions...
2018-11-16 16:10:55,858 | INFO : Adding Urey-Bradleys...
2018-11-16 16:10:55,858 | INFO : Adding improper torsions...
2018-11-16 16:10:55,859 | INFO : Adding CMAP torsions...
2018-11-16 16:10:55,859 | INFO : Adding trigonal angle terms...
2018-11-16 16:10:55,860 | INFO : Adding out-of-plane bends...
2018-11-16 16:10:55,860 | INFO : Adding pi-torsions...
2018-11-16 16:10:55,860 | INFO : Adding stretch-bends...
2018-11-16 16:10:55,861 | INFO : Adding torsion-torsions...
2018-11-16 16:10:55,861 | INFO : Adding Nonbonded force...


In [284]:
# create our alchemical state
factory = alchemy.AbsoluteAlchemicalFactory(consistent_exceptions=False)
alchemical_region = alchemy.AlchemicalRegion(alchemical_atoms=range(15),annihilate_sterics=True)
alchemical_atoms = list(alchemical_region.alchemical_atoms)

In [285]:
toluene_alchemical_system = factory.create_alchemical_system(
    reference_system=tol.system, alchemical_regions=alchemical_region)
alchemical_state = alchemy.AlchemicalState.from_system(toluene_alchemical_system)
# Create our custom State objects
# Need two different Thermodynamic State objects
# Context cache will grab correct thermodynamic state
# Keeping them in sync is in SamplerState.apply to context
# Have apply return accumulated work

alch_thermodynamic_state = ThermodynamicState(system=toluene_alchemical_system, temperature=temperature)
alch_thermodynamic_state = CompoundThermodynamicState(alch_thermodynamic_state, composable_states=[alchemical_state])
alch_thermodynamic_state.alchemical_atoms = alchemical_atoms
thermodynamic_state = ThermodynamicState(system=tol.system, temperature=temperature)
sampler_state = SamplerState(positions=tol.positions)

In [286]:
with open('test.pdb', 'w') as pdb:
    openmm.app.pdbfile.PDBFile.writeFile(tol.topology, tol.positions, pdb)

In [287]:
from blues.reporters import NetCDF4Reporter
filename = 'test.nc'
if os.path.exists(filename):
    os.remove(filename)
else:
    print("Sorry, I can not remove %s file." % filename)
nc_reporter = NetCDF4Reporter(filename, 1)
nc_reporter

In [288]:
ncmc_move = NCMCMove(timestep=timestep,
                 collision_rate=collision_rate,
                 n_steps=n_steps,
                 temperature=temperature,
                 reassign_velocities=True)
sampler = NCMCSampler(thermodynamic_state, 
                      alch_thermodynamic_state,
                      sampler_state,
                      move=ncmc_move, 
                      platform=None, reporter=nc_reporter, 
                      topology=tol.topology)
sampler.minimize(max_iterations=0)
sampler.run(n_iterations=50)

2018-11-16 16:11:09,577 | DEBUG : LocalEnergyMinimizer: platform is CUDA
2018-11-16 16:11:09,579 | DEBUG : Minimizing with tolerance 1.0 kcal/(A mol) and 0 max. iterations.
-12536.568855767953 kJ/mol
................................................................................
MCMC sampler iteration 0
<openmmtools.integrators.LangevinIntegrator; proxy of <Swig Object of type 'OpenMM::CustomIntegrator *' at 0x7f95e75dced0> >
2018-11-16 16:11:09,815 | DEBUG : NCMCMove: <openmmtools.integrators.LangevinIntegrator; proxy of <Swig Object of type 'OpenMM::CustomIntegrator *' at 0x7f95e75dced0> > 	 Context obtained, platform is CUDA
Step: 0 PE: -12537.818855767953 kJ/mol
Step: 1 PE: -12539.709480767953 kJ/mol
Step: 2 PE: -12539.779793267953 kJ/mol
Step: 3 PE: -12536.631355767953 kJ/mol
Step: 4 PE: -12530.287605767953 kJ/mol
'naccept'
<blues.integrators.AlchemicalExternalLangevinIntegrator; proxy of <Swig Object of type 'OpenMM::CustomIntegrator *' at 0x7f95e75dc9c0> >
2018-11-16 16:11:09,9

In [300]:
traj_nc = mdtraj.load_netcdf('test.nc', top='test.pdb')
traj_nc.image_molecules()

<mdtraj.Trajectory with 400 frames, 975 atoms, 321 residues, and unitcells at 0x7f9596b64da0>

In [301]:
view = nglview.show_mdtraj(traj_nc)
#view.camera = 'orthographic'
view.background = 'white'
# clear representations
#view.clear_representations()

# specify residue
view.add_licorice('ligand')
view.add_licorice('water')

view

NGLWidget(count=400)